## AI Server SDK for Python

### Install Dependencies

In [2]:
%pip install --upgrade ai-server-sdk
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


## Server Connection & Insights

### Creating the Server Connection

In [3]:
import os
from ai_server import ServerClient
from dotenv import load_dotenv

load_dotenv('../.env')
SECRET_KEY = os.getenv('DEV_SECRET_KEY')
ACCESS_KEY = os.getenv('DEV_ACCESS_KEY')

# Here we connect to the demo instance of CFG, but you would connect to your own instance
connection_url = 'https://workshop.cfg.deloitte.com/cfg-ai-dev/Monolith/api'

# This object creates a connection to the CFG server
server_connection = ServerClient(base = connection_url, access_key = ACCESS_KEY, secret_key = SECRET_KEY)

# Check if you are connected to the server
is_connected = server_connection.connected
print(f"Am I connected to the server? {is_connected}")

# Initilizing the server creates a new insight which is accessible through .cur_insight
my_insight = server_connection.cur_insight
print(f"This is my current insight ID: {my_insight}")

Am I connected to the server? True
This is my current insight ID: 8df5f2aa-b48b-4008-bfec-4fb5b3ce8e4a


### Create new insights

In [4]:
# You can create new insights by calling the .make_new_insight() method
new_insight = server_connection.make_new_insight() # This becomes your active insight

print(f"My current insight is now: {server_connection.cur_insight}")

My current insight is now: 5ec12e9a-1578-4303-a015-49d65dc3fa4e


### Listing open insights

In [5]:
my_open_insights = server_connection.get_open_insights()

print(f"These are my open insights: {my_open_insights}")

These are my open insights: ['5ec12e9a-1578-4303-a015-49d65dc3fa4e', '8df5f2aa-b48b-4008-bfec-4fb5b3ce8e4a']


### Dropping insights 

In [6]:
# You can drop all of your insights with the .drop_insights() method and passing in the list of insights
# Here we drop all of our open insight ids 
server_connection.drop_insights(my_open_insights) # This will create a new insight id since there must be an active insight

print(f"Here are my insight IDs after dropping: {server_connection.get_open_insights()}")

Here are my insight IDs after dropping: ['3ace0467-9956-46c8-ae16-94f851e9f3da']


### Python trick to check attributes

In [7]:
attributes = dir(server_connection)
# Find all of the attributes available to the server connection object
attributes = [attr for attr in attributes if not attr.startswith('__')]

print(f"Here are the attributes of the server connection object: {attributes}")

Here are the attributes of the server connection object: ['access_key', 'auth_headers', 'bearer_token', 'bearer_token_provider', 'connected', 'cookies', 'cur_insight', 'da_server', 'drop_insights', 'get_auth_headers', 'get_open_insights', 'get_openai_endpoint', 'get_partial_responses', 'get_pixel_output', 'import_data_product', 'loginBearerToken', 'loginUserAccessKey', 'logout', 'main_url', 'make_new_insight', 'monitors', 'open_insights', 'r', 'run_pixel', 'run_pixel_separate_thread', 'secret_key', 'send_request', 'upload_files']


## ModelEngine

### Basic Inferencing

In [8]:
from ai_server import ModelEngine

# An ID for the specific LLM you want to interact with
engine_id = os.getenv('DEV_LLM_CHAT_ENGINE_ID')

# We need to pass the engine id and an insight id to the ModelEngine object
llama3_model = ModelEngine(engine_id=engine_id, insight_id = server_connection.cur_insight)

question = "What weighs more, a pound of feathers or a pound of bricks?"

# The ask method sends a question to the model and returns the response
try:
    answer = llama3_model.ask(question)
except Exception as e:
    print(f"An error occured: {e}")

print(answer['response'])

print("We also get acess to: ")
print(f"messageId: {answer['messageId']}")
print(f"roomId: {answer['roomId']}")
print(f"numberOfTokensInPrompt", answer['numberOfTokensInPrompt'])
print(f"numberOfTokensInResponse", answer['numberOfTokensInResponse'])


They both weigh the same, 1 pound. The difference lies in their density and volume. A pound of feathers would occupy a larger volume than a pound of bricks due to the difference in their densities.
We also get acess to: 
messageId: 952e8e77-32d2-4bb8-a4bc-9ba1230d894d
roomId: 3ace0467-9956-46c8-ae16-94f851e9f3da
numberOfTokensInPrompt 12
numberOfTokensInResponse 36


### Advanced Inferencing with parameters

In [9]:
# We can add parameters to our requests such as context, history, max_new_tokens, repetition_penalty, seed, temperature, top_k, top_p, truncate, typical_p
params = {
    "temperature": 0.9, 
    "max_new_tokens": 200,
    "context": "You are a first grade teacher that uses language and explanations easy for children to understand."
    }

# Pass your parameters as a dictionary to the ask method using the param_dict argument
new_answer = llama3_model.ask(question, param_dict=params)

print(new_answer['response'])

They weigh the same, 1 pound. The feathers will take up more space than the bricks because feathers are very light and take up a lot of room, but they both weigh the same.


### Using chat history

In [10]:

# We can pass our own history to the ask method
# You can use as many dictionaries as you want in the history list but each one will add tokens to the request
history = [
    {"role": "user", "content": question},
    {"role": "assistant", "content": new_answer['response']}
]

# Here we change the context to a college professor and pass our chat history
params = {
    "temperature": 0.9, 
    "max_new_tokens": 200,
    "context": "You are college professor who provides complex answers backed by science.",
    "history": history
}

# Our new question references the chat history
new_question = "Can you explain your previous answer in more detail?"

new_answer = llama3_model.ask(new_question, params)

print(new_answer['response'])

Let's dive deeper into the concept of density and weight. Density is defined as the mass per unit volume of a substance. In the case of the feather and the brick, they both weigh 1 pound, which means they have the same mass.

However, the density of a feather is much lower than that of a brick. Feathers are essentially made up of air, which is very light. As a result, a large volume of feathers weighs the same as a small volume of bricks. This is why a pound of feathers takes up more space than a pound of bricks.

To quantify this, let's consider some approximate densities. The density of feathers is around 0.07-0.1 g/cm³, while the density of bricks is around 2-3 g/cm³. This means that for every cubic centimeter of feathers, you would need to have only about 0.07-0.1 grams to match the weight of a cubic centimeter of bricks.

Using these values, let's calculate the volume of a pound of feathers and a pound of bricks. Assuming a density of 0.09 g/cm³ for feathers, we can calculate the 

## VectorEngine

### Adding documents

In [11]:
from ai_server import VectorEngine

# Using a FAISS vector engine
vector_engine_id = os.getenv('DEV_VECTOR_ENGINE_ID')

# We initialize the VectorEngine object with the engine id and the current insight id
faiss_vector_engine = VectorEngine(engine_id=vector_engine_id, insight_id=server_connection.cur_insight)

file_path = "./ai-whitehouse.pdf"

# We can add documents to the faiss index with the addDocument method
faiss_vector_engine.addDocument(file_paths = [file_path])

True

### Listing uploaded documents

In [12]:
# Fetch a list of uploaded documents
my_documents = faiss_vector_engine.listDocuments()

print(my_documents)

[{'fileName': 'constitution.pdf', 'fileSize': 404.2470703125, 'lastModified': '2025-03-21 16:18:35'}, {'fileName': 'ai-whitehouse.pdf', 'fileSize': 578.16015625, 'lastModified': '2025-03-25 13:16:53'}]


### Performing a nearest neighbor search

In [13]:
query = "How does the document define machine learning?"

# Find the closest match(es) between the question bassed in and the embedded documents using Euclidena Distance.
nearest_neighbor = faiss_vector_engine.nearestNeighbor(search_statement=query, limit = 3, insight_id = server_connection.cur_insight)

for index, result in enumerate(nearest_neighbor):
    print(f"Result {index + 1}")
    print(f"SCORE: {result['Score']}")
    print(f"TOKENS: {result['Tokens']}")
    print(f"CONTENT: {result['Content']}")

Result 1
SCORE: 0.7842901945114136
TOKENS: 165
CONTENT: information, such as images, videos, audio clips, and text, that has been significantly modified or generated by algorithms, including by AI. (ff ) The term “testbed” means a facility or mechanism equipped for conducting rigorous, transparent, and replicable testing of tools and 11/15/23, 10:36 AM Executive Order on the Safe, Secure, and Trustworthy Development and Use of Artificial Intelligence | The White House https://www.whitehouse.gov/briefing-room/presidential-actions/2023/10/30/executive-order-on-the-safe-secure-and-trustworthy-development-and-u… 10/63 technologies, including AI and PETs, to help evaluate the functionality, usability, and performance of those tools or technologies.
Result 2
SCORE: 0.7212825417518616
TOKENS: 164
CONTENT: of Artificial Intelligence | The White House https://www.whitehouse.gov/briefing-room/presidential-actions/2023/10/30/executive-order-on-the-safe-secure-and-trustworthy-development-and-us… 6

### Removing documents

In [14]:
# Names of the files we want to remove
file_names = ['ai-whitehouse.pdf']

faiss_vector_engine.removeDocument(file_names = file_names)

print(faiss_vector_engine.listDocuments())

[{'fileName': 'constitution.pdf', 'fileSize': 404.2470703125, 'lastModified': '2025-03-21 16:18:35'}]


## DatabaseEngine

### Querying a database

In [15]:
from ai_server import DatabaseEngine

# An example H2 diabetes database
db_engine_id = os.getenv('DEV_DATABASE_ENGINE_ID')

# Connect to the database by passing the engine id and the current insight id
db = DatabaseEngine(engine_id=db_engine_id, insight_id=server_connection.cur_insight)

query = db.execQuery(query = "SELECT height, weight, location FROM diabetes WHERE height < 62 AND weight > 200")

print(query)

     LOCATION HEIGHT WEIGHT
0  Buckingham     61    256
1         Bjc     60    220
2      Louisa     61    203
3      Louisa     59    204
4      Louisa     61    220
5      Louisa     58    210


### Inserting data

In [16]:
insert = "INSERT INTO diabetes (height, weight, location) VALUES (65, 200, 'Rosslyn')"

# NOTE you will need to be an author or editor of the database to insert or delete data
db.insertData(query = insert)

query = db.execQuery(query = "SELECT height, weight, location FROM diabetes WHERE location = 'Rosslyn'")

print(query)

Exception: User does not have permission to exec query for this app

### Deleting data

In [ ]:
remove = "DELETE FROM diabetes WHERE location = 'Rosslyn'"

db.removeData(query = remove)

query = db.execQuery(query = "SELECT height, weight, location FROM diabetes WHERE location = 'Rosslyn'")

print(query)

Exception: User does not have permission to exec query for this app

## FunctionEngine

In [ ]:
from ai_server import FunctionEngine

# Weather function engine
weather_id = os.getenv('DEV_FUNCTION_ENGINE_ID')

function = FunctionEngine(engine_id=weather_id, insight_id=server_connection.cur_insight)

# Parameters will change based on the function you are using
output = function.execute({"lat":"37.540","lon":"77.4360"})

print(output)

Exception: prerna.engine.impl.storage.S3StorageEngine cannot be cast to prerna.engine.api.IFunctionEngine

## StorageEngine

### Listing Files

In [ ]:
from ai_server import StorageEngine

# Example S3 bucket
storage_engine_id = os.getenv('DEV_STORAGE_ENGINE_ID')

storageEngine = StorageEngine(engine_id = storage_engine_id, insight_id = server_connection.cur_insight)

s3_storage_path = "/my-new-test-folder/"

# A list of the files in the given path
my_dir = storageEngine.list(storagePath = s3_storage_path)

for file in my_dir:
    print (f"-- {file}")

# A list of details about the files in the given path
my_dir_details = storageEngine.listDetails(storagePath = '/my-new-test-folder/')

for file_details in my_dir_details:
    print (f"-- {file_details}")


-- machine-readable-business-employment-data-mar-2024-quarter.csv
-- version/
-- {'Path': 'machine-readable-business-employment-data-mar-2024-quarter.csv', 'Name': 'machine-readable-business-employment-data-mar-2024-quarter.csv', 'Size': 3583192.0, 'MimeType': 'text/csv; charset=utf-8', 'ModTime': '2024-06-13T19:11:49.768746813Z', 'IsDir': False, 'Tier': 'STANDARD', 'Metadata': {'atime': '2024-06-13T19:11:49.768746813Z', 'btime': '2024-11-01T21:28:04Z', 'content-type': 'text/csv; charset=utf-8', 'gid': '0', 'mode': '100640', 'mtime': '2024-06-13T19:11:49.768746813Z', 'tier': 'STANDARD', 'uid': '0'}}
-- {'Path': 'version', 'Name': 'version', 'Size': 0.0, 'MimeType': 'inode/directory', 'ModTime': '2000-01-01T00:00:00.000000000Z', 'IsDir': True}


## Running Pixels

In [ ]:
# You can use the active server connection to run pixels

simple_pixel_response = server_connection.run_pixel('1+1')

print('Simple Response --')
print(simple_pixel_response)

full_pixel_response = server_connection.run_pixel('1+1', full_response=True)

print('Full Response --')
print(full_pixel_response)

Simple Response --
2
Full Response --
{'insightID': 'f6839fb0-e65d-4cff-b539-821f72434c1d', 'pixelReturn': [{'pixelId': '27', 'pixelExpression': '1 + 1 ;', 'isMeta': False, 'timeToRun': 2, 'output': 2, 'operationType': ['OPERATION']}]}


### LLM Chat with pixels example

In [ ]:
engine_id = os.getenv('DEV_LLM_CHAT_ENGINE_ID')

chat = server_connection.run_pixel(
    f"LLM ( engine = [ '{engine_id}' ] , command = [ '<encode>What is the capital of Connecticut</encode>' ] , paramValues = [ {{ 'max_new_tokens' : 200 , 'temperature' : 0.3 }} ] )"
)
print(chat)

{'numberOfTokensInResponse': 6, 'numberOfTokensInPrompt': 136, 'response': 'The capital of Connecticut is Hartford.', 'messageId': 'cd299520-acf0-45a9-b02c-6c5d09c3a213', 'roomId': 'f6839fb0-e65d-4cff-b539-821f72434c1d'}
